In [ ]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

KeyboardInterrupt: 

## Data Fetching

In [ ]:
from utils.data_fetch import read_data
data = read_data()

In [ ]:
data

In [ ]:
data.info()

## Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [ ]:
X_train_full = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train_full =pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [ ]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [ ]:
X_train = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train=pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [ ]:
X_val = val_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_val=pd.concat([val_set["Spindle Speed"], val_set["Feed (mm/min)"]], axis=1)

In [ ]:
X_test = test_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_test=pd.concat([test_set["Spindle Speed"], test_set["Feed (mm/min)"]], axis=1)

In [ ]:
X_train.shape

In [ ]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [ ]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)

In [ ]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb],validation_data=(X_val,y_val))

In [ ]:
mse_score, rmse_score = model.evaluate(X_test,y_test)

In [ ]:
X_new = X_train[:3]

In [ ]:
X_train[:3]

In [ ]:
model.predict(X_new)

In [ ]:
y_train[:3]

### Model Tuning

In [ ]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [ ]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [ ]:
best_trial.summary()

In [ ]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [ ]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb, early_stopping_cb])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

In [ ]:
X_new

In [ ]:
Final_model.predict(X_new)

In [ ]:
y_train[:3]

In [ ]:
Final_model.save("Final_neural_network.h5")